In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Activation
from src.data_scripts import *
from src.modeling_scripts import *

In [2]:
raw_data = pd.read_csv('data/ca_energy_demand.csv')

In [3]:
clean_data = clean_data(raw_data)

/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 

In [4]:
# Normalizing data for LSTM model

data = normalize(clean_data)

In [5]:
data.describe()

,Demand,Year,Month,Day_of_Week,Hour,Normalized
count,45844.000000,45844.000000,45844.000000,45844.000000,45844.000000,45844.000000
mean,31758.887082,2017.605445,6.581232,3.000851,11.499106,0.031308
std,6525.024613,1.554086,3.388803,2.000038,6.922636,0.014445
min,17617.000000,2015.000000,1.000000,0.000000,0.000000,0.000000
25%,27019.000000,2016.000000,4.000000,1.000000,5.000000,0.020815
50%,30604.000000,2018.000000,7.000000,3.000000,11.000000,0.028751
75%,34411.000000,2019.000000,9.000000,5.000000,17.000000,0.037180
max,62787.000000,2020.000000,12.000000,6.000000,23.000000,0.100000


In [11]:
pred_date = '2020-01-29'
window_size = 8 * 7 * 24
weeks = 1
days = 7
hours = 24
num_pred = weeks * days * hours

In [12]:
X_train, y_train, X_test, y_test = prep_data_lstm(clean_data, pred_date, window_size, weeks=weeks, days=days, hours=hours)

In [13]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((38640, 1344, 1), (38640, 168), (1, 1344, 1), (1, 168))

In [14]:
n_features = 1

model = Sequential()
model.add(LSTM(1344, activation='relu', dropout=0.1, input_shape=(window_size, n_features)))
model.add(Dense(num_pred, activation='linear'))
model.compile(optimizer='adam', loss='mse')

In [82]:
model.fit(X_train, y_train, batch_size=128, epochs=5, verbose=1)

Epoch 1/5
302/302 [==============================] - 11614s 38s/step - loss: 1.8791e-04
Epoch 2/5
302/302 [==============================] - 11940s 40s/step - loss: 7.2487e-05
Epoch 3/5
302/302 [==============================] - 12148s 40s/step - loss: 5.6319e-05
Epoch 4/5
302/302 [==============================] - 12425s 41s/step - loss: 5.3351e-05
Epoch 5/5
302/302 [==============================] - 11916s 39s/step - loss: 5.2774e-05


In [83]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 1344)              7236096   
_________________________________________________________________
dense_5 (Dense)              (None, 168)               225960    
Total params: 7,462,056
Trainable params: 7,462,056
Non-trainable params: 0
_________________________________________________________________


In [84]:
yhat = model.predict(X_test, verbose=1)

1/1 [==============================] - 0s 1ms/step


In [85]:
unnorm_yhat = yhat * ((max_demand - min_demand)*10) + min_demand

In [86]:
unnorm_ytest = y_test * ((max_demand - min_demand)*10) + min_demand

In [88]:
mean_squared_error(unnorm_ytest, unnorm_yhat, squared=False)

2519.343734027062

In [90]:
unnorm_yhat

array([[26857.762, 25166.092, 24114.266, 23818.031, 24297.121, 25170.4  ,
        26218.455, 27559.652, 28641.121, 29681.836, 29401.986, 29605.46 ,
        29158.45 , 29252.506, 29280.34 , 29877.48 , 30718.535, 31414.637,
        32067.996, 32771.938, 32475.023, 32041.582, 30592.654, 28878.168,
        26825.809, 25304.107, 24241.258, 23901.1  , 24383.916, 25138.357,
        26376.646, 27927.387, 28792.254, 29360.02 , 29733.21 , 29685.14 ,
        29675.3  , 29497.947, 29813.945, 30220.562, 30885.613, 31889.688,
        32862.29 , 33294.42 , 33196.82 , 32533.621, 31254.348, 29292.492,
        27222.832, 25586.27 , 24527.627, 24362.633, 24472.406, 25476.168,
        26767.207, 27663.242, 28853.512, 29646.594, 30001.016, 29959.137,
        29782.639, 29651.605, 29821.277, 30476.059, 31147.219, 31885.297,
        32784.39 , 33454.445, 33140.605, 32461.309, 31083.652, 29393.912,
        27667.717, 25757.885, 24800.17 , 24433.334, 24915.24 , 25649.066,
        26837.102, 28051.436, 29236.41

In [91]:
unnorm_ytest

array([[26997., 25891., 25088., 24585., 24566., 25265., 27313., 30598.,
        31780., 30933., 29727., 28464., 27690., 27387., 27846., 28279.,
        29284., 30897., 33300., 34291., 33902., 33026., 31480., 29358.,
        27354., 26058., 25210., 24637., 24455., 25271., 27303., 30542.,
        31717., 30757., 29657., 28681., 28084., 27810., 27745., 28113.,
        29007., 30737., 33189., 34034., 33525., 32688., 31234., 29248.,
        27269., 26085., 25101., 24671., 24579., 25235., 27181., 30227.,
        31308., 30265., 29204., 28385., 27658., 27197., 27416., 28019.,
        28806., 30359., 32392., 33169., 32408., 31591., 30416., 28676.,
        26887., 25605., 24709., 24357., 24052., 24281., 25182., 26468.,
        26939., 26686., 26008., 25239., 24564., 24364., 24603., 25181.,
        26306., 27962., 30355., 31366., 30771., 30020., 29011., 27457.,
        25790., 24751., 24130., 23717., 23419., 23434., 24075., 24914.,
        25089., 25078., 25202., 24565., 24064., 23666., 23817., 